In [1]:
import numpy as np
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from models.detr import DETR, LossSegmentation
from models.segmentation import DETRsegm, PostProcessPanoptic
from models.matcher import HungarianMatcher
from hubconf import detr_resnet101_panoptic, detr_resnet3d_panoptic
from torchvision.transforms import Resize
from util.misc import get_world_size
import open3d as o3d
import torch.optim as optim
import pkbar

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
nib.imageglobals.logger.level = 51

In [3]:
def sort_func(path):
    path_id = int(path.split('/')[-1].split('_')[1])
    return path_id

In [4]:
def show_image_and_label(image, target):
    fig, axs = plt.subplots(nrows=1,ncols=2, squeeze=False,figsize=(12, 12))
    axs[0, 0].imshow(image)
    axs[0, 0].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Image')
    axs[0, 1].imshow(target)
    axs[0, 1].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Target')

In [ ]:
show_image_and_label()

In [5]:
def uniform_downsample_image(image, down_scale = 8):
    image_shape = image.shape
    dim_0_indexes = torch.arange(0, image_shape[0], down_scale)
    dim_1_indexes = torch.arange(0, image_shape[1], down_scale)
    dim_2_indexes = torch.arange(0, image_shape[2], down_scale)
    downsampled_image = image[dim_0_indexes,:,:]
    downsampled_image = downsampled_image[:,dim_1_indexes,:]
    downsampled_image = downsampled_image[:,:,dim_2_indexes]
    return downsampled_image

In [6]:
def downsample_image_to_given_size(image, size = 128):
    image_shape = image.shape
    dim_0_indexes = torch.arange(0, image_shape[0], int(image_shape[0]/size))
    dim_1_indexes = torch.arange(0, image_shape[1], int(image_shape[1]/size))
    dim_2_indexes = torch.arange(0, image_shape[2], int(image_shape[2]/size))
    downsampled_image = image[dim_0_indexes,:,:]
    downsampled_image = downsampled_image[:,dim_1_indexes,:]
    downsampled_image = downsampled_image[:,:,dim_2_indexes]
    return downsampled_image

In [7]:
def visualize_segmented_image(segmented_image):
#     segmented_image.squeeze()
    if segmented_image.get_device() > -1: segmented_image = segmented_image.to('cpu')
    classes = segmented_image.unique()
    colors = [[128,128,128],[255,0,0],[255,255,0],[0,255,0],[0,255,255],[0,0,255],[255,0,255]]
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    for _class in classes[1:]:
        points_numpy = (segmented_image == _class).nonzero().numpy()
        o3d_point_cloud = o3d.geometry.PointCloud()
        o3d_point_cloud.points = o3d.utility.Vector3dVector(points_numpy)
        o3d_point_cloud.estimate_normals()
        o3d_point_cloud.paint_uniform_color(np.array(colors[int(_class)-1])/255)
        vis.add_geometry(o3d_point_cloud)
    vis.run()
    vis.destroy_window()

In [35]:
def get_bounding_boxes(segmented_image):
    if segmented_image.get_device() > -1: segmented_image = segmented_image.to('cpu')
    classes = segmented_image.unique()
    bb_list = []
    for class_ in classes[1:]: 
        points = (segmented_image == class_).nonzero()
        x_min, x_max = points[:,0].min(), points[:,0].max()
        y_min, y_max = points[:,1].min(), points[:,1].max()
        z_min, z_max = points[:,2].min(), points[:,2].max()
        bb = torch.tensor([(x_max-x_min)/2, (y_max-y_min)/2, (z_max-z_min)/2, x_max-x_min, y_max-y_min, z_max-z_min])
        bb[0::2] = bb[0::2]/torch.tensor(segmented_image.shape)
        bb[1::2] = bb[1::2]/torch.tensor(segmented_image.shape)
        bb_list.append(bb)
    bbs = torch.stack(bb_list)
    return bbs

In [36]:
def get_masks(segmented_image):
    if segmented_image.get_device() > -1: segmented_image = segmented_image.to('cpu')
    classes = segmented_image.unique()
    masks_list = []
    for class_ in classes[1:]:
        mask = segmented_image.clone()
        mask[segmented_image == class_] = 1
        mask[segmented_image != class_] = 0
        masks_list.append(mask.short())
    masks = torch.stack(masks_list)
    return masks

In [37]:
def get_labels(segmented_image):
    return segmented_image.unique().long()[1:]

In [38]:
def create_target_dict(segmented_image):
    labels = get_labels(segmented_image)
    boxes = get_bounding_boxes(segmented_image)
    masks = get_masks(segmented_image)
    target_dict = {'labels': labels, 'boxes': boxes, 'masks': masks, 'seg_im': segmented_image}
    return target_dict

In [12]:
def get_box_corners(box):
    x_min = box[0] - box[3]/2
    x_max = box[0] + box[3]/2
    y_min = box[1] - box[4]/2
    y_max = box[1] + box[4]/2
    z_min = box[2] - box[5]/2
    z_max = box[2] + box[5]/2
    return torch.tensor([x_min,y_min,z_min,x_max,y_max,z_max])

In [13]:
def get_intersetion_box_corners(box_1,box_2):
    intersection_box_corners = torch.zeros(6)
    intersection_box_corners[0] = max(box_1[0],box_2[0])
    intersection_box_corners[1] = min(box_1[3],box_2[3])
    intersection_box_corners[2] = max(box_1[1],box_2[1])
    intersection_box_corners[3] = min(box_1[4],box_2[4])
    intersection_box_corners[4] = max(box_1[2],box_2[2])
    intersection_box_corners[5] = min(box_1[5],box_2[5])
    return intersection_box_corners

In [14]:
def compute_box_volume(box):
    box_volume = (box[3]-box[0])*(box[4]-box[1])*(box[5]-box[2])
    if box_volume < 0: box_volume = 0
    return box_volume

In [15]:
def compute_box_iou(box_1,box_2):
    box_1_corners = get_box_corners(box_1)
    box_2_corners = get_box_corners(box_2)
    intersection_box = get_intersetion_box_corners(box_1_corners,box_2_corners)
    intersection_box_volume = compute_box_volume(intersection_box)
    if intersection_box_volume == 0: return 0
    box_1_volume = compute_box_volume(box_1_corners) 
    box_2_volume = compute_box_volume(box_2_corners)
    iou = intersection_box_volume/(box_1_volume+box_2_volume-intersection_box_volume)
    return iou

In [16]:
def compute_average_precision(outputs, target, labels):
    ious = []
    for i, label in enumerate(labels):
        pred_box = outputs['pred_boxes'].squeeze()[i]
        target_box = target['boxes'][label] 
        iou = compute_box_iou(pred_box,target_box)
        ious.append(iou)
    ious_tensor = torch.tensor(ious)
    tp = (ious_tensor >= 0.5).count_nonzero()
    fn = abs(len(target['labels']) - 1 - tp)
    fp = abs((outputs['pred_logits'].squeeze().argmax(1) != 0).count_nonzero() - 1 - tp)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    return precision

In [53]:
compute_average_precision(outputs, aux_target, trgt_idx)

tensor(0.)

In [63]:
aux_target['boxes']

tensor([[0.1562, 0.1406, 0.0625, 0.3125, 0.7200, 0.3200],
        [0.1875, 0.1016, 0.1016, 0.3750, 0.5200, 0.5200],
        [0.3906, 0.2969, 0.1406, 0.7812, 1.5200, 0.7200],
        [0.1562, 0.2031, 0.1484, 0.3125, 1.0400, 0.7600],
        [0.2500, 0.2109, 0.1094, 0.5000, 1.0800, 0.5600],
        [0.2656, 0.2812, 0.1797, 0.5312, 1.4400, 0.9200],
        [0.3828, 0.3125, 0.1562, 0.7656, 1.6000, 0.8000]], device='cuda:0')

In [83]:
(outputs['pred_logits'].squeeze().argmax(1) != 0).count_nonzero()

tensor(8, device='cuda:0')

In [40]:
aux_dset = DatasetForSegmentation(image_paths,target_paths)
matcher = HungarianMatcher()

In [41]:
inpt = aux_dset[48]

aux_inpt = inpt[0].unsqueeze(0).unsqueeze(0).to(device)
aux_target = inpt[1]

for i in aux_target.items():
    aux_target[i[0]] = i[1].to(device)

outputs = detr(aux_inpt)

/home/francisco/workspace/CHD_Classifier_by_Francisco_Lourenço/models/position_encoding.py:84: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


In [43]:
src_idx, trgt_idx = matcher(outputs,[aux_target])[0]
src_idx, trgt_idx

(tensor([0, 1, 2, 3, 4, 6, 7]), tensor([2, 5, 3, 1, 0, 6, 4]))

In [72]:
outputs['pred_boxes'], aux_target['boxes']

(tensor([[[0.4499, 0.4627, 0.5484, 0.4836, 0.4890, 0.4989],
          [0.4548, 0.4529, 0.5035, 0.4869, 0.4908, 0.5009],
          [0.4627, 0.4683, 0.5292, 0.4746, 0.4836, 0.4798],
          [0.4580, 0.4595, 0.5244, 0.4915, 0.4897, 0.4887],
          [0.4598, 0.4540, 0.5158, 0.4949, 0.4821, 0.4874],
          [0.4710, 0.4554, 0.5220, 0.4867, 0.4888, 0.4816],
          [0.4614, 0.4634, 0.5132, 0.4775, 0.4843, 0.4850],
          [0.4582, 0.4520, 0.5345, 0.4850, 0.4959, 0.4764]]], device='cuda:0',
        grad_fn=<SelectBackward0>),
 tensor([[0.1562, 0.1406, 0.0625, 0.3125, 0.7200, 0.3200],
         [0.1875, 0.1016, 0.1016, 0.3750, 0.5200, 0.5200],
         [0.3906, 0.2969, 0.1406, 0.7812, 1.5200, 0.7200],
         [0.1562, 0.2031, 0.1484, 0.3125, 1.0400, 0.7600],
         [0.2500, 0.2109, 0.1094, 0.5000, 1.0800, 0.5600],
         [0.2656, 0.2812, 0.1797, 0.5312, 1.4400, 0.9200],
         [0.3828, 0.3125, 0.1562, 0.7656, 1.6000, 0.8000]], device='cuda:0'))

In [17]:
def compute_panoptic_quality(pred_seg, target):
    ious = []
    for i, label in enumerate(target['labels'][1:]):
        target_mask = target['masks'][i] 
        pred_mask = (pred_seg == label).int()
        pred_mask[pred_mask == 0] = -1
        intersection = (pred_mask == target_mask).count_nonzero()
        union = pred_mask[pred_mask == 1].count_nonzero() + target_mask[target_mask == 1].count_nonzero() - intersection
        iou = intersection/union
        ious.append(iou)
    ious_tensor = torch.tensor(ious)
    tp = (ious_tensor > 0.5).count_nonzero()
    fn = abs(len(target['labels']) - 1 - tp)
    fp = abs(len(pred_seg.unique()) - 1 - tp)
    if tp > 0:
        sq = ious_tensor[ious_tensor > 0.5].sum() / tp
        rq = rq = tp / (tp + fp/2 + fn/2)
        pq = sq*rq
    else:
        sq = 0
        rq = 0
        pq = 0
    return pq, sq, rq

In [18]:
class DatasetForSegmentation(Dataset):
    
    def __init__(self, image_paths, target_paths, down_scale=8):
        self.image_paths = image_paths
        self.target_paths = target_paths
        self.down_scale = down_scale
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, i):
        image_np_array = nib.load(self.image_paths[i]).get_fdata()
        image_torch_tensor = torch.from_numpy(image_np_array).float()
        downsampled_image = uniform_downsample_image(image_torch_tensor, down_scale=self.down_scale)
#         downsampled_image = downsample_image_to_given_size(image_torch_tensor, size=64)
        target_np_array = nib.load(self.target_paths[i]).get_fdata()
        target_torch_tensor = torch.from_numpy(target_np_array)
        target_torch_tensor[target_torch_tensor > 7] = 0
        downsampled_target = uniform_downsample_image(target_torch_tensor, down_scale=self.down_scale)
#         downsampled_target = downsample_image_to_given_size(target_torch_tensor, size=64)
        target_dict = create_target_dict(downsampled_target)
        return downsampled_image, target_dict

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [20]:
path = '/home/francisco/workspace/ImageCHD_dataset'

In [21]:
image_paths = glob(f'{path}/*image.nii.gz',recursive=True)
target_paths = glob(f'{path}/*label.nii.gz',recursive=True)

In [22]:
image_paths.sort(key=sort_func)
target_paths.sort(key=sort_func)

In [23]:
ds = DatasetForSegmentation(image_paths,target_paths)

In [24]:
detr_seg = detr_resnet3d_panoptic()
# detr_seg.eval();
# detr_seg.half();
detr_seg.to(device);

In [24]:
detr_seg = detr_resnet3d_panoptic()
detr = detr_seg.detr
detr.to(device);

In [25]:
# aux_dset = DatasetForSegmentation(image_paths,target_paths)

In [26]:
# inpt = aux_dset[2]

# aux_inpt = inpt[0].unsqueeze(0).unsqueeze(0).to(device)
# aux_target = inpt[1]

# for i in aux_target.items():
#     aux_target[i[0]] = i[1].to(device)

# outputs = aux(aux_inpt)

In [34]:
# outputs = detr_seg(image_reshaped)

In [33]:
matcher = HungarianMatcher()
loss_segmentation = LossSegmentation(matcher)

In [36]:
# loss = loss_segmentation.loss_boxes(outputs, [aux_target])

In [38]:
# loss

In [22]:
is_thing_map = {'0': False, '1': True, '2': True, '3': True, '4': True, '5': True, '6': True, '7': True}
post_process_panoptic = PostProcessPanoptic(is_thing_map)

In [23]:
lr = 0.0001
n_epoch = 25
weight_decay = 10e-4
ACC_Threshold = 0.02

In [24]:
optimizer = optim.AdamW(detr_seg.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
torch.autograd.set_detect_anomaly(True)
train_dset_size = 87
down_scale = 4

valid_dset = DatasetForSegmentation(image_paths[train_dset_size:],target_paths[train_dset_size:], down_scale=down_scale)
for epoch in range(n_epoch):
    kbar = pkbar.Kbar(target=(train_dset_size+len(valid_dset)-2), epoch=epoch, num_epochs=n_epoch, width=16, stateful_metrics=["Accuracy"])
    running_loss_t = 0.0
    running_loss_v = 0.0
    rdm = torch.randperm(train_dset_size)
    train_image_paths = [image_paths[i] for i in rdm]
    train_target_paths = [target_paths[i] for i in rdm]
    train_ds = DatasetForSegmentation(train_image_paths, train_target_paths, down_scale=down_scale)

    for idx, b in enumerate(train_ds):
        image = b[0]#.half()
        image = image.unsqueeze(0).unsqueeze(0).to(device)
        target = b[1]
        for i in target.items():
#             if i[0] in {'boxes'}:
#                 target[i[0]] = i[1].half()
            target[i[0]] = i[1].to(device)
            
        outputs = detr_seg(image)
        optimizer.zero_grad()
        
#         loss = loss_segmentation.loss_masks(outputs, [target]) #To train masks
        loss = loss_segmentation.loss_boxes(outputs, [target]) #To train boxes
        loss['loss_mask'].backward()
        optimizer.step()
        running_loss_t = loss['loss_mask'].item()
        kbar.update(idx, values=[("Train Loss", running_loss_t)])

    kbar.add(1, values=[("Validation Loss", 0), ("Accuracy", 0)])        
    with torch.no_grad():
        acc = 0
        for ids,b in enumerate(valid_dset):
            image = b[0].unsqueeze(0).unsqueeze(0).to(device)
            target = b[1]
            for i in target.items():
                target[i[0]] = i[1].to(device)

            outputs = detr_seg(image)
#             loss = loss_segmentation.loss_masks(outputs, [target]) #To train masks
            loss = loss_segmentation.loss_boxes(outputs, [target]) #To train boxes
            running_loss_v = loss['loss_mask'].item()
            src_idx, trgt_idx = matcher(outputs,[target])[0]
            pred_seg = post_process_panoptic(outputs[trgt_idx],[tuple(torch.tensor(b[0].shape).tolist())]).to(device)
            pq,sq,rq = compute_panoptic_quality(pred_seg, target)
            kbar.update(idx+ids, values=[("Validation Loss", running_loss_v), ("PQ", 100*pq),("SQ", 100*sq),("RQ", 100*rq)])
print('Finished Training')

Epoch: 1/25


/home/francisco/workspace/CHD_Classifier_by_Francisco_Lourenço/models/position_encoding.py:84: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


In [29]:
# torch.save(detr_seg.state_dict(), '/home/francisco/workspace/CHD_Classifier_by_Francisco_Lourenço/chackpoint_lr_10e-4.pth')

***

# Visualization:

In [53]:
is_thing_map = {'0': False, '1': True, '2': True, '3': True, '4': True, '5': True, '6': True, '7': True}

In [54]:
post_process_panoptic = PostProcessPanoptic(is_thing_map)

In [35]:
pred_masks = outputs['pred_masks']

In [42]:
result = post_process_panoptic(outputs,[tuple(torch.tensor(image.squeeze().shape).tolist())])

In [61]:
visualize_segmented_image(aux_target['seg_im'])

In [60]:
aux_inpt[0,0]

torch.Size([64, 64, 25])